####  Finding optimal hyperparameters for one of the algorithms (LightGBM in this case)

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

import tensorflow as tf
import keras
from keras import layers

#### Load the data

In [5]:
df = pd.read_csv("processed_regression_housing.csv")
df.head()

,housing_median_age,total_rooms,population,median_income,median_house_value,distance_to_nearest_city,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY
0,52.0,1274.0,558.0,5.6431,341300.0,17.06,0,0,0,1
1,52.0,1627.0,565.0,3.8462,342200.0,17.06,0,0,0,1
2,52.0,919.0,413.0,4.0368,269700.0,17.06,0,0,0,1
3,52.0,2535.0,1094.0,3.6591,299200.0,16.55,0,0,0,1
4,52.0,3104.0,1157.0,3.1200,241400.0,16.55,0,0,0,1


####  Same X/y + train/test split

In [6]:
categorical_variables = [ 
    "ocean_proximity_<1H OCEAN",
    "ocean_proximity_INLAND",
    "ocean_proximity_ISLAND",
    "ocean_proximity_NEAR BAY"]

# continuous variables also into a list
continuous_variables = [ 
    "housing_median_age",
    "total_rooms",
    "population",
    "median_income",
    "distance_to_nearest_city",
    "median_house_value"]

# the usual X/y -split
X = df.drop("median_house_value", axis=1)
y = df['median_house_value']

# usual train/test -split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# NOTE! SCALING => some of the algorithms require this
scaler = StandardScaler()

# create separate versions for the scaled data
# because we need both unscaled and scaled versions later
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

####  In this example, use RandomizedSearchCV to search better hyperparameters for our algorithm

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
from scipy.stats import randint

# Define the parameter distributions with integer ranges
# Notice how we use NumPy and scipy in order to have ranges of values
# for the RandomizedSearch, which it tries to combine in a random manner 
# => based on luck, you might stumble upon a very good combination of parameters
param_dist = {
    'learning_rate': np.linspace(0.08, 0.135, 30), 
    'num_leaves': randint(25, 51),  
    'max_depth': randint(-1, 35), 
}

# Setup RandomizedSearchCV
# search 300 times (n_iter)
# n_jobs => -1 => use all CPU cores
# cv = cross-validation strategy (higher is often better, but takes more time), typical values 3-5
random_search = RandomizedSearchCV(
    estimator=lgb.LGBMRegressor(), 
    param_distributions=param_dist, 
    n_iter=300,  # Number of random combinations to test
    cv=3,
    n_jobs=-1,
    verbose=1,  
    scoring='neg_mean_squared_error'
)

# Fit the randomized search
random_search.fit(X_train, y_train)

# Get the best parameters
best_params_random = random_search.best_params_
print("\nBest parameters from RandomizedSearchCV:", best_params_random)

Fitting 3 folds for each of 300 candidates, totalling 900 fits
